In [53]:
%%capture --no-stderr
!pip install -U langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python

In [54]:
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from typing import List
import tavily

In [55]:
class MaterialResponse(BaseModel):
    material: List[str]

In [56]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [57]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [58]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [59]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

In [60]:
tool.invoke("material requires for highway construction in india")

[{'url': 'https://constrofacilitator.com/types-of-waste-materials-used-for-road-construction-in-india/',
  'content': 'Facebook Instagram Linkedin Pinterest RSS Share Twitter Youtube To construct India’s first Pavement Quality Concrete (PQC) with steel slag, the National Highways Authority of India (NHAI) started trials for its possible use in road construction.The authority permitted Central Road Research Institute (CRRI) to construct a 1-km-long trial patch in the Raigarh district, for PQC of the Panvel-Indapur section of NH-66 near Mumbai, where 100 percent natural aggregates were replaced by steel slag-derived aggregates. In February 2023, in its effort to increase the use of plastic waste in road construction, the road transport and highways ministry made the use of the waste material in hot mix bituminous wearing coat or top layer mandatory on all service and slip roads throughout the National Highway (NH) network in the country.'},
 {'url': 'https://stellartesthouse.com/what-are

In [61]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [62]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
today = datetime.datetime.today().strftime("%D")

prompt = ChatPromptTemplate(
    [
        ("system", f"You are an expert construction planner with access to a vast knowledge base and real-time web search capabilities. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)


llm_with_tools = llm.bind_tools([tool])
llm_chain = prompt | llm_with_tools
@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = tool_chain.invoke("What materials are needed for highway construction in India?")
print(response)


content='In addition to the primary materials listed above, highway construction in India may also require other materials and components, including:\n\n12. **Asphalt Mixes**: Different types of asphalt mixes, such as dense bitumen macadam (DBM) and bituminous concrete (BC), are used based on traffic loads and environmental conditions.\n\n13. **Sealant and Joint Fillers**: Materials like silicone or polyurethane sealants are used to seal joints and cracks in asphalt and concrete pavements.\n\n14. **Emulsions**: Bitumen emulsions are used for surface treatments, tack coats, and as a binding agent in cold mixtures.\n\n15. **Reinforced Soil Materials**: Such as mechanically stabilized earth (MSE) walls that use reinforcement materials to increase load-bearing capacity.\n\n16. **Surveying Instruments**: These include levels, total stations, and GPS devices for accurate planning and alignment of the highway.\n\n17. **Safety Barriers**: Materials used for constructing concrete or metal crash

In [63]:
response.content

'In addition to the primary materials listed above, highway construction in India may also require other materials and components, including:\n\n12. **Asphalt Mixes**: Different types of asphalt mixes, such as dense bitumen macadam (DBM) and bituminous concrete (BC), are used based on traffic loads and environmental conditions.\n\n13. **Sealant and Joint Fillers**: Materials like silicone or polyurethane sealants are used to seal joints and cracks in asphalt and concrete pavements.\n\n14. **Emulsions**: Bitumen emulsions are used for surface treatments, tack coats, and as a binding agent in cold mixtures.\n\n15. **Reinforced Soil Materials**: Such as mechanically stabilized earth (MSE) walls that use reinforcement materials to increase load-bearing capacity.\n\n16. **Surveying Instruments**: These include levels, total stations, and GPS devices for accurate planning and alignment of the highway.\n\n17. **Safety Barriers**: Materials used for constructing concrete or metal crash barrier

In [67]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# class MaterialBase(BaseModel):
#     material: str
#     price: int
    
# class MaterialResponse(BaseModel):
#     data: list[MaterialBase]

# prompt = ChatPromptTemplate(
#     [
#         ("system", """You are an expert construction planner with access to a vast knowledge base and real-time web search capabilities. The response must be in JSON format as follows:{{
#             "material": [
#                 "Material 1",
#                 "Material 2",
#                 "Material 3",
#                 "... (other materials)"
#             ]
#         }}"""),
#         ("human", "{user_input}"),
#         ("placeholder", "{messages}"),
#     ]
# )

class MaterialBase(BaseModel):
    material: str
    price: int

class MaterialResponse(BaseModel):
    data: list[MaterialBase]

prompt = ChatPromptTemplate(
    [
         ("system", """You are an expert construction planner with access to a vast knowledge base and real-time web search capabilities. The response must be in below JSON format strictly without any explanation and comments:
        {{
            "data": [
                {{
                    "material": "Material 1", 
                    "price": 1000
                }},
                {{
                    "material": "Material 2", 
                    "price": 1500
                }},
                {{
                    "material": "Material 3", 
                    "price": 2000
                }}
                // Add more materials and their prices as needed
            ]
        }}"""),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)


llm_with_tools = llm.bind_tools([tool])
llm_chain = prompt | llm_with_tools
parser = PydanticOutputParser(pydantic_object=MaterialResponse)

@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    return response

    

response = tool_chain.invoke("What materials are needed for metro constructions in India and give the price in per meter cube?")
print(response)

content='{\n    "data": [\n        {\n            "material": "Cement",\n            "price": 6000\n        },\n        {\n            "material": "Steel (TMT Bars)",\n            "price": 50000\n        },\n        {\n            "material": "Concrete (Ready Mix Concrete)",\n            "price": 7000\n        },\n        {\n            "material": "Sand",\n            "price": 1500\n        },\n        {\n            "material": "Aggregates",\n            "price": 1200\n        },\n        {\n            "material": "Bricks",\n            "price": 5000\n        }\n    ]\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 1036, 'total_tokens': 1172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed

In [68]:
parsed_response = parser.parse(response.content)


In [69]:
parsed_response.model_dump()

{'data': [{'material': 'Cement', 'price': 6000},
  {'material': 'Steel (TMT Bars)', 'price': 50000},
  {'material': 'Concrete (Ready Mix Concrete)', 'price': 7000},
  {'material': 'Sand', 'price': 1500},
  {'material': 'Aggregates', 'price': 1200},
  {'material': 'Bricks', 'price': 5000}]}

In [3]:
from pydantic import BaseModel, create_model

# Example parsed response
parsed_response = {
    'data': [
        {'material': 'Cement', 'price': 6000},
        {'material': 'Steel (TMT Bars)', 'price': 50000},
        {'material': 'Concrete (Ready Mix Concrete)', 'price': 7000},
        {'material': 'Sand', 'price': 1500},
        {'material': 'Aggregates', 'price': 1200},
        {'material': 'Bricks', 'price': 5000}
    ]
}

# Dynamically generate fields for the Pydantic model
fields = {item['material']: (int, ...) for item in parsed_response['data']}

# Create a Pydantic model dynamically
DynamicMaterial = create_model("DynamicMaterial", **fields)

# Example usage
material_data = {
    "Cement": 6000,
    "Steel (TMT Bars)": 50000,
    "Concrete (Ready Mix Concrete)": 7000,
    "Sand": 1500,
    "Aggregates": 1200,
    "Bricks": 5000
}

# Validate data using the dynamically created model
material_instance = DynamicMaterial(**material_data)
print(material_instance.model_dump())


{'Cement': 6000, 'Steel (TMT Bars)': 50000, 'Concrete (Ready Mix Concrete)': 7000, 'Sand': 1500, 'Aggregates': 1200, 'Bricks': 5000}
